# 萝卜机器人 算法组 集成代码

## 目录

- 视频/图片 输入
- Tinypose 输出
- 格式转换 输入
- 3d base line 输出
- 格式转换 输入
- bvh 输出

## 环境预备

In [ ]:
# clone some necessary files
!sh "./initialize.sh" 

### Colab

如果使用 Colab 就需要运行下面的代码。

In [ ]:
# 占用显存
import torch
# about 1.1 GB
a = torch.Tensor([1000, 1000, 1000]).cuda()

# 查看显卡信息
!nvidia-smi

In [ ]:
# 挂载谷歌云盘 (是否需要？)
# from google.colab import drive
# drive.mount('/content/drive/')

# clone repo if necessary
import os
os.chdir("/content/")
!git clone https://github.com/IsshikiHugh/RadishRobotCV.git

### Tinypose


In [ ]:
import os
%pip install paddlepaddle-gpu
import paddle
paddle.utils.run_check()
print(paddle.__version__)

In [ ]:
# install pd
%cd "/content/RadishRobotCV/PaddleDetection"
%pip install -r requirements.txt
!python setup.py install
!python ppdet/modeling/tests/test_architectures.py

In [ ]:
# # usage

# %cd "/content/RadishRobotCV/PaddleDetection"

# # single picture
# !python3 deploy/python/det_keypoint_unite_infer.py --det_model_dir=output_inference/picodet_s_192_pedestrian --keypoint_model_dir=output_inference/tinypose_128x96 --device=GPU --image_file=1.jpg

# %cd "/content/RadishRobotCV/PaddleDetection"
# # vedio
# !python3 deploy/python/det_keypoint_unite_infer.py --det_model_dir=output_inference/picodet_s_192_pedestrian --keypoint_model_dir=output_inference/tinypose_128x96 --device=GPU  --save_res=True --video_file=1.mp4

### 3d Base Line

In [ ]:
%pip install h5py tensorflow imageio numpy

In [ ]:
# # useage
# %cd "/content/RadishRobotCV/3dbaseline"
# !python src/openpose_3dpose_sandbox.py --camera_frame --residual --batch_norm --dropout 0.5 --max_norm --evaluateActionWise --use_sh --epochs 1 --load 24371 --pose_estimation_json <path> --gif_fps 24


## 使用

在`RadishRobotCV/input_dir`下放入`input_video.mp4`

In [ ]:
%cd "/content/RadishRobotCV"
%mkdir "input_dir"
%mkdir "input_dir/input_jsonDir_2d"

### video - json

In [ ]:
%cd "/content/RadishRobotCV"
%mkdir "./PaddleDetection/input_vedio"
%cp "./input_dir/input_video.mp4" "./PaddleDetection/input_vedio/input_video.mp4"

%cd "/content/RadishRobotCV/PaddleDetection"
!python deploy/python/det_keypoint_unite_infer.py --det_model_dir=output_inference/picodet_s_192_pedestrian --keypoint_model_dir=output_inference/tinypose_128x96 --device=GPU --video_file=input_vedio/input_video.mp4 --save_res=True

%cp "det_keypoint_unite_video_results.json" "/content/RadishRobotCV/input_dir/input_json.json" 

### json(tinypose) - json(3dbaseline)

In [ ]:
%cd "/content/RadishRobotCV"
from transcribe_helper import tinypose23dbl
import json

%cd "/content/RadishRobotCV"
tJson = json.load(open("./input_dir/input_json.json","r"))
output = tinypose23dbl.transcribePP23D(tJson=tJson)

filepath = "./input_dir/input_jsonDir_2d"

print(type(output))
output = json.dumps(output)
datas = json.loads(output)

cnt = -1
for data in datas.items():
    cnt += 1
    _out_file = filepath+'/{0}.json'.format(str(cnt))
    with open(_out_file, 'w') as outfile:
            json.dump(data, outfile, indent=4) 
# Write to files in Frames  "/content/RadishRobotCV/input_dir/input_jsonDir_2d"

### json - json

In [ ]:
%cd "/content/RadishRobotCV/3dbaseline"
!python src/openpose_3dpose_sandbox.py --camera_frame --residual --batch_norm --dropout 0.5 --max_norm --evaluateActionWise --use_sh --epochs 1 --load 24371 --pose_estimation_json /content/RadishRobotCV/input_dir/input_jsonDir_2d --gif_fps 24

%cp "./maya/outputData.json" "/content/RadishRobotCV/input_dir/input_json2.json"

### json - numpy

In [ ]:
%cd "/content/RadishRobotCV"
from transcribe_helper import json2npy
import numpy as np
#TODO: make dir to json file
keyPointsData_JSON = json.load(open("./input_dir/input_json2.json","r"))
outputNpy = json2npy.transcribeJSON2NPY(keyPointsData_JSON)
fileName = "output.npy"
np.save(fileName, outputNpy)
%mv output.npy "./input_dir/input.npy"

### numpy - bvh

In [ ]:
from transcribe_helper import transcribe_toBVH
%cd "/content/RadishRobotCV"
customPose = np.load("./input_dir/input.npy")
output = './output.bvh'
transcribe_toBVH.makeBvhWithNpyFile(customPose, output)